In [1]:
%matplotlib inline


# Corpora and Vector Spaces

텍스트를 벡터 공간에 represent 하는 실습<br>
그리고 corpus 스트리밍과 corpus가 저장소에 존재하는 다양한 포맷에 대해서<br>

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

9개의 document로 이루어 진 corpus

## From Strings to Vectors

이번 document는 string

In [3]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

이 9개의 documents로 이루어진 작은 corpus는 각각 한 문장으로 이루어져 있습니다.<br>

먼저, documents를 토큰화 하고, 간단히 불용어와 corpus에서 한번나온 단어를 제거 하겠습니다. 

In [4]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# 단어를 소문자로 바꾸고 불용어 처리
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# 한번만 나온 단어 제거
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


documents를 처리하는 방식은 다양할 수 있습니다.

documents를 처리하는 방식은 앱이나 언어 어디에 의존하냐에 따라 다양해 지는데, gensim은 특성 방식으로 제한을 두지 않았습니다.

대신에, document는 표면적인게 아닌 document에서 추출된 특징으로 represent했습니다.

어떻게 특징을 추출할지는 사용자에게 달려 있습니다.

앞으로 일반적인 방식의 접근법을 볼겁니다(bag-of-words).

그러나 언제나 다른 앱의 도메인에서는 다른 특징을 갖는다는 것을 기억해야 합니다.

document를 변환시키기 위해서 bag-or-words를 사용할 것입니다.

- Question: How many times does the word `system` appear in the document?
- Answer: Once.

위의 형식을 띄는 질-답 쌍입니다.

질의는 integer형 id로 represent되고 id와 질의는 딕셔너리로 맵핑될 것입니다.

In [5]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('./tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

2021-09-06 16:09:44,204 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-09-06 16:09:44,204 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-09-06 16:09:44,261 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2021-09-06T16:09:44.205960', 'gensim': '4.0.1', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2021-09-06 16:09:44,261 : INFO : Dictionary lifecycle event {'fname_or_handle': './tmp/deerwester.dict', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-09-06T16:09:44.261589', 'gensim': '4.0.1', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bi

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


gensim 딕셔너리 클래스를 이용해서 corpus의 고유id와 모든 단어를 매칭시켰습니다.<br>

In [6]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [7]:
from itertools import chain

mylist = [[1,2,3,4],[5,6,7,8],[9,10,11,12]]

print(f'처리 전 : {mylist}')

mylist_merged = list(chain.from_iterable(mylist))

print(f'처리 후: {mylist_merged}')

처리 전 : [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]
처리 후: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


토큰화된 documents를 벡터들로 변환시키기 위해서:

In [8]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # 딕셔너리에 없는 interaction 이란 단어는 무시되었다

[(0, 1), (1, 1)]


doc2bow 함수는 단순히 모든 단어의 빈도수를 세고, 단어를 integer id로 변한하고, 희소 벡터 형태로 값을 반환합니다.

In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

2021-09-06 16:09:44,320 : INFO : storing corpus in Matrix Market format to ./tmp/deerwester.mm
2021-09-06 16:09:44,321 : INFO : saving sparse matrix to ./tmp/deerwester.mm
2021-09-06 16:09:44,322 : INFO : PROGRESS: saving document #0
2021-09-06 16:09:44,323 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2021-09-06 16:09:44,324 : INFO : saving MmCorpus index to ./tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


## Corpus Streaming -- One Document at a Time

In [10]:
from smart_open import open  # for transparently opening remote files


class MyCorpus:
    def __iter__(self):
        for line in open('https://radimrehurek.com/mycorpus.txt'):
            # 한 document에 한 줄씩, 공백일 기준으로 나뉘어 반환
            yield dictionary.doc2bow(line.lower().split())

Gensim은 corpus가 iterable한 객체면 사용 가능합니다. like list,numpy,pandas

이런 유연함이 우리 corous 클래스가 디스크,네트워크,데이터베이스 등에서 직접 stream으로 documents를 가져올 수 있게 만듭니다.<br>
Gensim에 있는 모델들은 모든 벡터가 한번에 메모리에 한번에 올라가도록 하지 않습니다. <br>

우리가 사용할 파일이 어떻게 생겼는지는 중요하지 않다.<br>
단지 __iter__이 입력 형식하고 얼마나 잘 맞는지가 중요하다.(xml 파싱 같은)<br>
입력데이터를 잘 파싱해서 각 document를 토큰들의 list로 만드는 것 뿐이다.<br>
그리고 토큰들을 그들의 id와 매칭시켜 딕셔너리로 만들고 __iter__ 함수 내에서 희소벡터를 yield하면 된다.<br>

In [11]:
corpus_memory_friendly = MyCorpus()  # corpus를 메모리에 올리지 않습니다!!!!
print(corpus_memory_friendly)

In [12]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


딕셔너리도 파이썬 list처럼 메모리 절약적으로 사용할 수 있습니다.

In [13]:
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('https://radimrehurek.com/mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stoplist
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

2021-09-06 16:09:44,933 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-09-06 16:09:44,934 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)
2021-09-06 16:09:44,935 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)", 'datetime': '2021-09-06T16:09:44.935894', 'gensim': '4.0.1', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


## Corpus Formats

Gensim에서는 corpus를 스트리밍하게 사용 할 수 있는 Matrix Market 같은 다양한 저장 포멧이 있습니다.

In [14]:
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize('./tmp/corpus.mm', corpus)

2021-09-06 16:09:44,957 : INFO : storing corpus in Matrix Market format to ./tmp/corpus.mm
2021-09-06 16:09:44,959 : INFO : saving sparse matrix to ./tmp/corpus.mm
2021-09-06 16:09:44,959 : INFO : PROGRESS: saving document #0
2021-09-06 16:09:44,960 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2021-09-06 16:09:44,961 : INFO : saving MmCorpus index to ./tmp/corpus.mm.index


Joachim's SVMlight format <http://svmlight.joachims.org/><br>
Blei's LDA-C format <http://www.cs.princeton.edu/~blei/lda-c/><br>
GibbsLDA++ format <http://gibbslda.sourceforge.net/><br>

In [15]:
corpora.SvmLightCorpus.serialize('./tmp/corpus.svmlight', corpus)
corpora.BleiCorpus.serialize('./tmp/corpus.lda-c', corpus)
corpora.LowCorpus.serialize('./tmp/corpus.low', corpus)

2021-09-06 16:09:44,971 : INFO : converting corpus to SVMlight format: ./tmp/corpus.svmlight
2021-09-06 16:09:44,975 : INFO : saving SvmLightCorpus index to ./tmp/corpus.svmlight.index
2021-09-06 16:09:44,976 : INFO : no word id mapping provided; initializing from corpus
2021-09-06 16:09:44,976 : INFO : storing corpus in Blei's LDA-C format into ./tmp/corpus.lda-c
2021-09-06 16:09:44,977 : INFO : saving vocabulary of 2 words to ./tmp/corpus.lda-c.vocab
2021-09-06 16:09:44,979 : INFO : saving BleiCorpus index to ./tmp/corpus.lda-c.index
2021-09-06 16:09:44,980 : INFO : no word id mapping provided; initializing from corpus
2021-09-06 16:09:44,980 : INFO : storing corpus in List-Of-Words format into ./tmp/corpus.low
2021-09-06 16:09:44,981 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2021-09-06 16:09:44,981 : INFO : saving LowCorpus index to ./tmp/corpus.low.index


반대로 Matrix Market 파일에서 corpus를 불러올 수도 있습니다

In [16]:
corpus = corpora.MmCorpus('./tmp/corpus.mm')

2021-09-06 16:09:44,988 : INFO : loaded corpus index from ./tmp/corpus.mm.index
2021-09-06 16:09:44,989 : INFO : initializing cython corpus reader from ./tmp/corpus.mm
2021-09-06 16:09:44,991 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


Corpus stream 들은 바로 print 할 수없습니다.

In [17]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [18]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

[[(1, 0.5)], []]


or



In [19]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]


두번째 방법이 좀 더 메모리 친화적이고, 첫번째 방법은 간편합니다.<br>
<br>
Matrix Market document stream을 Blei's LDA-C 포맷으로 저장하려면 다음과 같이 합니다.<br>

In [20]:
corpora.BleiCorpus.serialize('./tmp/corpus.lda-c', corpus)

2021-09-06 16:09:45,049 : INFO : no word id mapping provided; initializing from corpus
2021-09-06 16:09:45,050 : INFO : storing corpus in Blei's LDA-C format into ./tmp/corpus.lda-c
2021-09-06 16:09:45,052 : INFO : saving vocabulary of 2 words to ./tmp/corpus.lda-c.vocab
2021-09-06 16:09:45,053 : INFO : saving BleiCorpus index to ./tmp/corpus.lda-c.index


## Compatibility with NumPy and SciPy

Gensim에는 또한 벡터를 numpy와 scipy로 변환할 수있는 기능을 제공합니다.

In [21]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5, 2])  # random matrix as an example
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
number_of_corpus_features = 10 
numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)

 `scipy.sparse` 매트릭스 들로 변환

In [22]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5, 2)  # random sparse matrix as example
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)